<a href="https://colab.research.google.com/github/mahekagarwalla/SolarMapping/blob/main/WeatherPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Section 1:Import Libraries

In [ ]:
import requests
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor
from sklearn.metrics import mean_squared_error
from datetime import datetime , timedelta
import pytz




In [ ]:
API_KEY= '429f98c9c72a9fda44e061ee958c2f2d'
BASE_URL= 'https://api.openweathermap.org/data/2.5/'

1. Fetch Current Weather Data

In [ ]:
def get_current_weather(city):
  BASE_URL = 'https://api.openweathermap.org/data/2.5/'
  API_KEY = '429f98c9c72a9fda44e061ee958c2f2d'
  url=f"{BASE_URL}weather?q={city}&appid={API_KEY}&units=metric"
  response=requests.get(url)
  data= response.json()
  return {
      'city': data['name'],
      'current_temp':round(data['main']['temp']),
      'feels_like':round(data['main']['feels_like']),
      'temp_min':round(data['main']['temp_min']),
      'temp_max':round(data['main']['temp_max']),
      'humidity':round(data['main']['humidity']),
      'humidity':round(data['main']['humidity']),
      'description':(data['weather'][0]['description']),
      'country': (data['sys']['country']),
      'wind_gust_dir': data['wind']['deg'],
      'pressure':data['main']['pressure'],
      'WindGustSpeed': data['wind']['speed']

  }

2. Read Historical Data

In [ ]:
def read_historical_data(filename):
  df = pd.read_csv(filename)
  df=df.dropna()
  df=df.drop_duplicates()
  return df


3. Prepare data for training

In [ ]:
from re import X
def prepare_data(data):
  le= LabelEncoder()
  data['WindGustDir']=le.fit_transform(data['WindGustDir'])
  data['RainTomorrow']=le.fit_transform(data['RainTomorrow'])

  X=data[['MinTemp','MaxTemp','WindGustDir','WindGustSpeed','Humidity','Pressure', 'Temp']]
  y=data['RainTomorrow']

  return X,y,le




4. Train Rain Prediction Model

In [ ]:
def train_rain_model(x,y):
  x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)
  model = RandomForestClassifier(n_estimators=100, random_state=42)
  model.fit(x_train,y_train)

  y_pred=model.predict(x_test)

  print("Mean Squared Error for Rain Model")

  print(mean_squared_error(y_test, y_pred))

  return model





5. Prepare regression data

In [ ]:
def prepare_regression_data(data, feature):
  x,y= [],[]

  for i in range(len(data)-1):
    x.append(data[feature].iloc[i])

    y.append(data[feature].iloc[i+1])

  x= np.array(x).reshape(-1,1)
  y= np.array(y)
  return x,y




Train Regression Madel

In [ ]:
def train_regression_model(x,y):
  model = RandomForestRegressor(n_estimators=100, random_state=42)
  model.fit(x,y)
  return model

Predict Future

In [ ]:
def predict_future(model, current_value):
  predictions= [current_value]

  for i in range(5):
    next_value= model.predict(np.array([[predictions[-1]]]))


    predictions.append(next_value[0])

  return predictions[1:]


Weather Analysis Function

In [ ]:
def weather_view():
  city= input('Enter any city name: ')
  current_weather = get_current_weather(city)

  historical_data= read_historical_data('/content/weather.csv')

  x,y,le = prepare_data(historical_data)

  rain_model = train_rain_model(x, y)

  wind_deg= current_weather ['wind_gust_dir'] % 360
  compass_points =  [
        ("Ν", 0, 11.25), ("NNE", 11.25, 33.75), ("NE", 33.75, 56.25),
        ("ENE", 56.25, 78.75), ("E", 78.75, 101.25), ("ESE", 101.25, 123.75),
        ("SE", 123.75, 146.25), ("SSE", 146.25, 168.75), ("S", 168.75, 191.25),
        ("SSW", 191.25, 213.75), ("SW", 213.75, 236.25), ("WSW", 236.25, 258.75),
        ("W", 258.75, 281.25), ("WNW", 281.25, 303.75), ("NW", 303.75, 326.25),
        ("NNW", 326.25, 348.75),
      ]

  compass_direction= next((point for point, start, end in compass_points if start <= wind_deg < end),"N")

  compass_direction_encoded = le.transform([compass_direction])[0] if compass_direction in le.classes_ else -1

  current_data = {
      'MinTemp': current_weather['temp_min'],
      'MaxTemp': current_weather['temp_max'],
      'WindGustDir': compass_direction_encoded,
      'WindGustSpeed': current_weather['WindGustSpeed'],
      'Humidity': current_weather['humidity'],
      'Pressure': current_weather['pressure'],
      'Temp': current_weather['current_temp'],
  }
  current_df = pd.DataFrame([current_data])


  rain_prediction = rain_model.predict(current_df)[0]


  x_temp , y_temp = prepare_regression_data(historical_data , 'Temp')
  x_hum,y_hum = prepare_regression_data(historical_data, 'Humidity')
  temp_model= train_regression_model(x_temp,y_temp)
  hum_model= train_regression_model(x_hum, y_hum)


  future_temp = predict_future(temp_model, current_weather['temp_min'])
  future_humidity = predict_future(hum_model, current_weather['humidity'])

  timezone = pytz.timezone('Asia/Karachi')
  now = datetime.now(timezone)
  next_hour= now+ timedelta(hours=1)
  next_hour= next_hour.replace(minute=0, second=0, microsecond=0)

  future_times= [(next_hour+ timedelta(hours=i)).strftime("%H:00") for i in range(5)]

  print(f"City: {city}, {current_weather['country']}")
  print(f"Current Temperature:  {current_weather['current_temp']}")
  print(f"Feels Line:  {current_weather['feels_like']}")
  print(f"Minimum Temperature:  {current_weather['temp_min']}")
  print(f"Maximum Temperature:  {current_weather['temp_max']}")
  print(f"Humidity:  {current_weather['humidity']}")
  print(f"Weather Prediction:  {current_weather['description']}")
  print(f"Rain Prediction: {'Yes'if rain_prediction else 'NO'}")

  print("\n Future Temperature Prediction:")

  for time,temp in zip(future_times, future_temp):
    print(f"{time}:{round(temp,1)}")

  print("\n Future Temperature Prediction:")

  for time,humidity in zip(future_times, future_humidity):
    print(f"{time}:{round(humidity,1)}")

weather_view()











Enter any city name: mumbai 
Mean Squared Error for Rain Model
0.1506849315068493
City: mumbai , IN
Current Temperature:  28
Feels Line:  31
Minimum Temperature:  28
Maximum Temperature:  28
Humidity:  65
Weather Prediction:  clear sky
Rain Prediction: Yes

 Future Temperature Prediction:
11:00:27.7
12:00:22.8
13:00:25.4
14:00:24.7
15:00:23.4

 Future Temperature Prediction:
11:00:56.2
12:00:45.2
13:00:46.3
14:00:48.2
15:00:53.7
